In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_current

AS
WITH main AS (
    SELECT DISTINCT
        a.promotion_id,
        a.bonus_buy_id,
        a.bonus_buy_note,
        a.bonus_buy_profile,
        a.promo_start_date,
        a.promo_end_date,
        a.store_id,
        a.buy_item_id,
        a.discount_product_id,
        a.value_claimback,
        a.form_claimback,
        CASE
            WHEN a.bonus_buy_profile = 'ZB01' AND a.promo_start_date - 7 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.promo_end_date THEN 1
            ELSE 0
        END block_sto,
        CASE
            WHEN a.bonus_buy_profile = 'ZB01' AND a.promo_start_date - 7 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.promo_end_date - 7 THEN 1
            WHEN a.bonus_buy_profile = 'ZB10' AND a.promo_start_date - 18 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.promo_start_date - 7 THEN 1
            ELSE 0
        END block_po,
        'STORE_SKU' `level`
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_promotion a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b USING (store_id)
WHERE
    b.store_status_desc = 'Đang hoạt động'
    AND b.business_unit IN ("1500", "2000")
    AND b.is_win_plus <> 'YES'
    AND (
        (
            -- Free gift
            a.bonus_buy_status IS NULL
            AND a.bonus_buy_profile = 'ZB01'
            AND LEFT(a.bonus_buy_note, 1) = 'B'
            AND (a.promo_start_date - 7 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.promo_end_date)
        )
        OR
        (
            -- Single item discount
            IFNULL(a.bonus_buy_status, '0') IN ('0', '1')
            AND a.bonus_buy_profile = 'ZB10'
            AND (a.promo_start_date - 18 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.promo_start_date - 7)
        )
    )
)
SELECT *
FROM main;

""")